# Accessing and using BARRA2 and BARPA data for research at NCI

#### AMOS 2024 Workshop Landing Page 

https://opus.nci.org.au/display/DAE/AMOS+2024+Workshop%3A++Accessing+and+using+BARRA2+and+BARPA+data+for+research

## BARRA2

Bureau of Meteorology Atmospheric high-resolution Regional Reanalysis for 
Australia - Version 2 (BARRA2) is a reanalysis from 1979 to the present day covering Australia, New Zealand and a portion of South-East Asia.

NCI Data Catalogue: https://doi.org/10.25914/1x6g-2v48

Extended Documentation: https://opus.nci.org.au/x/DgDADw

## BARPA

The Bureau of Meteorology Atmospheric Regional Projections for Australia (BARPA) delivers high-resolution dynamical downscaling of CMIP6 experiments over CORDEX-Australasia and Australian domains.

NCI Data Catalogue: https://doi.org/10.25914/z1x6-dq28

Extended Documentation: https://opus.nci.org.au/x/CQDADw

## Accessing BARRA2 & BARPA

The outputs for BARRA2 and BARPA are store at NCI in the ob53 and py18 projects respectively.
Access to these projects can be requested in the usual way at the NCI account management pages - https://my.nci.org.au/mancini
The files are also available for direct download from NCI's Thredds server:
- [BARRA2](https://dap.nci.org.au/thredds/remoteCatalogService?catalog=https://dapds00.nci.org.au/thredds/catalogs/ob53/catalog.xml)
- [BARPA](https://dap.nci.org.au/thredds/remoteCatalogService?catalog=https://dapds00.nci.org.au/thredds/catalogs/py18/catalog.xml)

### List of variables

A list of the variables used by BARPA and BARRA2 can be found [here](https://github.com/joshuatorrance/barpa-barra2-amos2024/blob/main/BARRA2_BARPA_variable_list.csv).

***
This notebook covers some basic interactions with BARPA and BARRA2 data and some simple manipulations using *xarray*.

For more information on the multitude of tools available with *xarray* check out the xarray documentation:
https://docs.xarray.dev/en/stable/getting-started-guide/index.html
***

## Enter the notebook directory

To begin, navigate to the directory containing this notebook. The 'nci_ipynb' package, developed by NCI, is designed to automate this process. 
For more details, please refer https://pypi.org/project/nci-ipynb

In [ ]:
import os
import nci_ipynb

os.chdir(nci_ipynb.dir())
print(os.getcwd())

First we load the required python modules and start a dask client to speed up our computation.

In [ ]:
# Imports for the notebook
import os, sys
from glob import glob
from datetime import datetime
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
import cartopy.crs
import dask.distributed

Let's explictly load dask so we can check progress

In [ ]:
# Copy and paste the dashboard link/path from this cell's output
# to the Dask tab on the left.
client = dask.distributed.Client()
client

There should now be a dask client running (click on "Launch dashboard in JupyterLab" to see the dashboard). The dask client will allow for better parallelisation of xarray operations such as opens multiple files or processing large datasets.

You can see the progress of dask operations in the dask dashboard.

## 1. Loading BARRA2/BARPA data - the basic way

We will build a path to the BARRA2 data manually.

In [ ]:
## Data location
# Let's define the path to the files we're interested in
# BARRA2
source_id = 'BARRA-R2'
domain_id = 'AUS-11'   # Domain of BARRA-R2, in native grid 0.11deg
RCM_institution_id = 'BOM'  
driving_source_id = 'ERA5'  # Driving global model
driving_experiment_id = 'historical'  
driving_variant_label = 'hres'   # Using ERA5 HRES analysis

barra2_top_path = "/g/data/ob53/BARRA2/output/reanalysis/"

barra_r2_root_path = f"{barra2_top_path}/{domain_id}/{RCM_institution_id}/" + \
        f"{driving_source_id}/{driving_experiment_id}/{driving_variant_label}/{source_id}/v1"

Similarly we can also build a path to the BARPA data.

In [ ]:
## Data location
# BARPA
source_id = 'BARPA-R'
domain_id = 'AUS-15'   # Domain of BARPA-R, in native grid 0.154deg
RCM_institution_id = 'BOM'  
# Driving CMIP6 model from ACCESS-CM2, ACCESS-ESM1-5, CESM2, CMCC-ESM2, EC-Earth3, MPI-ESM1-2-HR, NorESM2-MM
# Or ERA5 (evaluation)
driving_source_id = 'ACCESS-CM2' 
driving_experiment_id = 'historical'  #  CMIP6 scenarios: historical, ssp370, ssp126
driving_variant_label = 'r4i1p1f1'  # CMIP6 model ensemble member

barpa_top_path = "/g/data/py18/BARPA/output/CMIP6/DD/"

barpa_r_root_path = f"{barpa_top_path}/{domain_id}/{RCM_institution_id}/" + \
        f"{driving_source_id}/{driving_experiment_id}/{driving_variant_label}/{source_id}/v1-r1"

Next we specify the variable, and time period

In [ ]:
### Pick the root path that interests you, either BARPA or BARRA2, comment out the other one. ###
root_path = barra_r2_root_path
# root_path = barpa_r_root_path

## freq - Time resolution of the data
# e.g. BARRA - mon, day, 3hr, 1hr
#      BARPA - mon, day, 6hr, 1hr
freq = "1hr"

## variable_id
# Choose the variable to look at, e.g. "ts" for surface temperature
variable_id = "ts"

## Date (YYYYMM)
# With glob we can use wild cards to find the files we want
# e.g. "2014??" for all of 2014 or "20140[123]" for the first three months
# Note that BARRA2 data has one file per month, BARPA data has one file per year
date = "2014??"

# Build a string to use with glob
glob_str = os.path.join(root_path, freq, variable_id, "*", f"*{date}.nc")

# Pass the string to glob will will return a matching list of file paths
file_list = sorted(glob(glob_str))

Now that we have the file paths, open the dataset using xarray

In [ ]:
# If we have a single file we can use open_dataset
#ds = xr.open_dataset(file_list[0])

# If we have a list of files we use open_mfdataset
# Use parallel=True to take advantage of Dask's multiprocessing
ds = xr.open_mfdataset(file_list, parallel=True)

# Quick look
ds

In [ ]:
# More information about this variable and data
ds[variable_id].attrs

## 2. Loading BARRA2/BARPA data with Intake-ESM

Instead of constructing the file paths by hand, you can use NCI Intake-ESM catalog files to manipulate BARRA2/BARPA data collections.

For more details in NCI indexing scheme, please refer https://opus.nci.org.au/display/DAE/Dataset+catalogue+Indexes+and+Intake


In [ ]:
import intake

# Specify the Intake-esm catalog files.
catalog_files = {"BARPA":"/g/data/dk92/catalog/v2/esm/barpa-py18/catalog.json",
                "BARRA2":"/g/data/dk92/catalog/v2/esm/barra2-ob53/catalog.json"}

# Choose the collection name
collection_name = "BARRA2" # BARPA

# Read in the catalogue which we will later use to interrogate the content
data_catalog = intake.open_esm_datastore(catalog_files[collection_name])

Next, we interrogate the catalogue using keys

But what are the keys?  
(We have already encountered before!)

In [ ]:
list(data_catalog.df.keys())

We can search the catalogue based on a group of identifier keys.

In [ ]:
# In this example, we query for a listing of all the 
#     one-hourly "ts" (surface temperature) during 2014

# Set up the query
query = dict(
    variable_id=["ts"],
    time_range=["2014/*"],
    freq=["1hr"],
)

# perform the search
catalog_subset = data_catalog.search(**query)

# The information from the searched catalog object could be viewed in a pandas table.
print(catalog_subset.df)

In [ ]:
# You could also set keywords when loading the dataset.
dsets = catalog_subset.to_dataset_dict(
#    cdf_kwargs={'chunks':{'lat': 646, 'lon': 1082, 'time':100}}
)
dsets

In [ ]:
# You could also set keywords when loading the dataset.
dsets = catalog_subset.to_dataset_dict(
    cdf_kwargs={'chunks':{'lat': 646, 'lon': 1082, 'time':100}}
)
dsets

## 3. Loading BARRA2/BARPA data using ESMLOADER

'esmloader' is a module included with these notebooks to simplify access to BARPA and BARRA2 datasets



In [ ]:
import esmloader
import importlib

importlib.reload(esmloader)

# Specify the which data collection you want to load, i.e. BARRA2 or BARPA.
collection_name = "BARRA2"
barra2 = esmloader.EsmCat(collection_name)

In [ ]:
help(barra2.load_barra2_data)

In [ ]:
# Load some BARRA data
ds = barra2.load_barra2_data("BARRA-R2", "1hr", "ts", tstart=2014010100, tend=2015010100)
ds

In [ ]:
importlib.reload(esmloader)
barpa = esmloader.EsmCat("BARPA")

# Load some BARPA data
ds = barpa.load_barpa_data("BARPA-R", "ACCESS-CM2", "historical", "1hr", "ts", tstart=2014010100, tend=2014060100)
ds

### Load data contains some other helpful functions
Here's a couple of them

In [ ]:
# Examine a particular variable
_ = barra2.whatis('1hr', 'pr')

In [ ]:
# List the available variables
_ = barra2.list_barra2_variables('BARRA-R2', '1hr')

## Instantaneous vs. Accumulated variables
The variables used in BARPA and BARRA2 can be separated into two group, *instantaneous* and *accumulated*.

*Instantaneous* variables give a snapshot of the underlying model stat at the given time.

*Accumulated* variables give an aggregate view of a given time window (e.g. hourly mean, daily max). Accumulated variables will have an additional coordinate, 'time_bnds'.

It's important to note that time values are different between instantaneous and accumulated variables.
For instantaneous variables the time value match the start of the window when snapshot was taken, i.e. 00:00, 01:00.
Accumulated variables use time values in the centre of their window, i.e. 00:30, 01:30.

It's important to keep these different time values in mind if one if combining variables in some way, e.g. performing arithmetic or plotting.

In [ ]:
variable_id = 'tas'
freq = '1hr'

# What is this variable?
barra2.whatis(freq, variable_id)

# Take a look at the first time step
ds_inst = barra2.load_barra2_data("BARRA-R2", freq, variable_id, tstart=2014010100, tend=2015010100)
ds_inst['time'][0].data

In [ ]:
variable_id = 'tasmax'
freq = '1hr'

# What is this variable?
barra2.whatis(freq, variable_id)

# Take a look at the first time step
ds_accum = barra2.load_barra2_data("BARRA-R2", freq, variable_id, tstart=2014010100, tend=2015010100)
ds_accum['time'][0].data

In [ ]:
# Accumulated variables have time_bnds
ds_accum['time_bnds'][0:3].compute()

## Indexing and Plotting Data
Xarray has sophisticated indexing tools available.
There are many ways to index data with Xarray, below are a couple of examples.

See Xarray's [documentation](https://docs.xarray.dev/en/latest/user-guide/indexing.html) for more details.

Xarray data sets can be easily plotting with matplotlib.

In [ ]:
variable_id = 'ts'
ds = barra2.load_barra2_data("BARRA-R2", "1hr", variable_id, tstart=2014010100, tend=2015010100)
ds

### First timestep

In [ ]:
# Select the first timestep using the index
ds_first_timestep = ds.isel(time=0)
ds_first_timestep

In [ ]:
# Alternatively select the first timestep by giving a string
# Look what happens if we don't specify said string precisely
ds.sel(time='2014-01-01')

In [ ]:
# Alternatively select the first timestep by giving a string
# Look what happens if we don't specify said string precisely
ds_first_timestep = ds.sel(time='2014-01-01T00:00')
ds_first_timestep

### Basic Plotting
xarray uses matplotlib to allow for quick and convenient plotting.

In [ ]:
# Plot the first field

# Can only plot data arrays (not datasets)
da = ds_first_timestep[variable_id]
da.plot()

In [ ]:
# Plot the first field - with coastlines!

# Can only plot data arrays (not datasets)
da = ds_first_timestep[variable_id]

# Build a cartopy projection so we can draw on the coastlines
centre_lon = da['lon'].mean().values
projection = cartopy.crs.PlateCarree(central_longitude=centre_lon)

# Now plot the field with the transform.
plot = da.plot(
    transform=cartopy.crs.PlateCarree(),
    subplot_kws={"projection": projection})

# Draw the coastlines using cartopy
plot.axes.coastlines()

### More indexing - Zoom in on Melbourne

In [ ]:
# Select the Melbourne region using slice
melb_lat, melb_lon = -37.840935, 144.946457
width = 1.5

ds_melb = ds.sel(lat=slice(melb_lat - width/2, melb_lat + width/2),
                 lon=slice(melb_lon - width/2, melb_lon + width/2))

In [ ]:
# Let's convert the temperature to centigrade
ds_melb[variable_id] = ds_melb[variable_id] - 273.25

# The above arithmetic will not preserve the DataArray's attributes
# So let's copy them here and update the units.
ds_melb[variable_id].attrs = ds[variable_id].attrs
ds_melb[variable_id].attrs['units'] = 'C'

In [ ]:
# Let's plot the resulting data array as we did before
# We can reuse the project we defined earlier
da = ds_melb.isel(time=0)[variable_id]

plot = da.plot(
    transform=cartopy.crs.PlateCarree(),
    subplot_kws={"projection": projection})

plot.axes.coastlines()

### Data Manipulation - Mean temperature in Melbourne during 2014

In [ ]:
# Take our Melbourne dataset and caculate the mean of each spatial field then plot the result
ds_melb[variable_id].mean(dim=['lat', 'lon']).plot()

# xarray uses matplotlib to handle the plotting
# Add a custom title to the plot using the standard matplotlib command
plt.title(f"Mean {da.attrs['long_name']} in Melbourne region ({da.attrs['units']})")

### Exercises
1. Plot the average temperature in Melbourne by time-of-day
2. Plot the min and max daily temperatures in Melbourne

In [ ]:
# First get time in the the local Melb timezone
# Use pandas to add the UTC timezone to 'time', convert it to Melbourne's timezone (AEDT), then remove the timezone again
time_melb = pd.to_datetime(ds_melb['time']).tz_localize('UTC').tz_convert('Australia/Melbourne').tz_localize(None)

# Replace time with time_melb
ds_melb_aedt = ds_melb.assign_coords(time_melb=("time", time_melb)).drop('time').swap_dims({'time': 'time_melb'})

# Add the attributes for time_melb so our plots below are nicer
ds_melb_aedt['time_melb'].attrs = {'standard_name': 'time_melb', 'axis': 'T', 'long_name': 'Time in AEDT'}

ds_melb_aedt

#### Average temperature by time of day

In [ ]:
# Now plot the mean spatial field value, averaged for each hour of the day
ds_melb_aedt[variable_id].mean(dim=['lat', 'lon']).groupby("time_melb.hour").mean().plot()

plt.title(f'Mean {ds_melb_aedt[variable_id].attrs["long_name"]} by time of day in Melbourne during 2014')

plt.xlabel(f'Hour of the day (Melbourne timezone)')
plt.ylabel(f'Mean {ds_melb_aedt[variable_id].attrs["long_name"]} ({ds_melb_aedt[variable_id].attrs["units"]})')

#### Min and Max Temperature

In [ ]:
# Daily min and max is usually from 9am to 9am
# Resample our hourly data into daily data, and offset to 9am
ds_melb_aedt.resample({'time_melb': '1D'}, offset='+9H').max().max(dim=['lat', 'lon'])[variable_id].plot(label='Daily Max')
ds_melb_aedt.resample({'time_melb': '1D'}, offset='+9H').mean().mean(dim=['lat', 'lon'])[variable_id].plot(label='Daily Mean')
ds_melb_aedt.resample({'time_melb': '1D'}, offset='+9H').min().min(dim=['lat', 'lon'])[variable_id].plot(label='Daily Min')

# Let's add a legend and title, etc.
plt.legend(bbox_to_anchor=(1.01, 1.0))

plt.title(f'{ds_melb_aedt[variable_id].attrs["long_name"]} in Melbourne')

plt.xlabel(f'{ds_melb_aedt["time_melb"].attrs["long_name"]}')
plt.ylabel(f'{ds_melb_aedt[variable_id].attrs["long_name"]} ({ds_melb_aedt[variable_id].attrs["units"]})')

In [ ]:
client.close()